<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/american_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal as mnorm

def callBS(s, K, T, sigma, r, q):
    if s <= 0:
        return 0
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)


def callDelta(s, K, T, sigma, r, q):
    if s <= 0:
        return 0
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.cdf(d1)


def Dmin(K, deltaT, r):
    return (1 - np.exp(-r * deltaT)) * K


def criticalS(D, K, deltaT, sigma, r):
    def f(s):
        return s - K - callBS(s - D, K, deltaT, sigma, r, 0)

    def fprime(s):
        return 1 - callDelta(s - D, K, deltaT, sigma, r, 0)

    return fsolve(f, x0=1.5 * K, fprime=fprime).item()


def binorm(a, b, corr):
    cov = [[1, corr], [corr, 1]]
    return mnorm.cdf(x=[a, b], mean=[0, 0], cov=cov)


def AmericanCall(D, S, K, t, deltaT, sigma, r):
    # t = time until dividend
    # deltaT = time between dividend and maturity
    Z0 = S - np.exp(-r * t) * D
    if D <= Dmin(K, deltaT, r):
        return callBS(Z0, K, t + deltaT, sigma, r, 0)
    Sstar = criticalS(D, K, deltaT, sigma, r)
    Zstar = Sstar - D
    d1 = (np.log(Z0 / Zstar) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    d1prime = (np.log(Z0 / K) + (r + 0.5 * sigma ** 2) * (t + deltaT)) / (
        sigma * np.sqrt(t + deltaT)
    )
    d2prime = d1prime - sigma * np.sqrt(t + deltaT)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    corr = -np.sqrt(t / (t + deltaT))
    M1 = binorm(-d1, d1prime, corr)
    M2 = binorm(-d2, d2prime, corr)
    return (
        Z0 * (N1 + M1)
        + np.exp(-r * t) * (D - K) * N2
        - np.exp(-r * (t + deltaT)) * K * M2
    )

In [ ]:
# example parameters

S = 50
K = 55
D = 2
t = 1           # time until dividend
deltaT = 1      # time between dividend and maturity
sigma = 0.4
r = 0.02


print('The critical price is', round(criticalS(D, K, deltaT, sigma, r),2), end='.\n')
print('It is optimal to exercise if and only if S_t is great than this price, where S_t denotes the underlying price just before it goes ex-dividend.')

The critical price is 98.72.
It is optimal to exercise if and only if S_t is great than this price, where S_t denotes the underlying price just before it goes ex-dividend.


In [ ]:
print('The value of the American option is', AmericanCall(D, S, K, t, deltaT, sigma, r).round(2), end='.')

The value of the American option is 9.0.